In [ ]:
# 코랩에서 Chrome과 ChromeDriver 설치
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# 필요한 패키지 설치
!pip install selenium pandas

# 환경 변수 설정
import os
os.environ['PATH'] += ':/usr/bin'


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,056 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,024 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packa

In [ ]:
# 코랩용 크롤링 코드 (수정 버전)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import os

def crawl_seoul_sinkhole_colab():
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-plugins')
    chrome_options.add_argument('--disable-images')
    chrome_options.add_argument('--remote-debugging-port=9222')

    # 수정된 WebDriver 생성 방법
    try:
        # 방법 1: Service 객체 사용 (권장)
        service = Service('/usr/bin/chromedriver')
        driver = webdriver.Chrome(service=service, options=chrome_options)
    except:
        try:
            # 방법 2: 직접 경로 없이 생성
            driver = webdriver.Chrome(options=chrome_options)
        except:
            # 방법 3: 환경변수 설정 후 생성
            os.environ['PATH'] += ':/usr/bin'
            driver = webdriver.Chrome(options=chrome_options)

    try:
        print("브라우저 시작 중...")

        # 페이지 로드
        url = "https://sciencesay.shinyapps.io/sinkhole/"
        driver.get(url)
        print("페이지 로드 완료")

        # 페이지 로딩 대기
        wait = WebDriverWait(driver, 20)

        # 테이블이 로드될 때까지 대기
        print("테이블 로딩 대기 중...")
        wait.until(EC.presence_of_element_located((By.ID, "DataTables_Table_0")))

        # 서울특별시 필터링 적용
        print("서울특별시 필터링 적용 중...")
        search_box = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="DataTables_Table_0_filter"]/label/input')))
        search_box.clear()
        search_box.send_keys("서울특별시")
        time.sleep(3)

        # 데이터 저장 리스트
        all_data = []
        page_num = 1
        max_pages = 50  # 무한루프 방지

        while page_num <= max_pages:
            print(f"페이지 {page_num} 크롤링 중...")

            try:
                # 현재 페이지의 테이블 행들 가져오기
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#DataTables_Table_0 tbody tr")))
                rows = driver.find_elements(By.CSS_SELECTOR, "#DataTables_Table_0 tbody tr")

                # 각 행의 데이터 추출
                page_data_count = 0
                for row in rows:
                    try:
                        cells = row.find_elements(By.TAG_NAME, "td")
                        if cells and len(cells) >= 9:
                            row_data = {
                                'id': cells[0].text,
                                'acc_date': cells[1].text,
                                'acc_loc': cells[2].text,
                                'acc_reason': cells[3].text,
                                'width': cells[4].text,
                                'length': cells[5].text,
                                'depth': cells[6].text,
                                'geo_feat': cells[7].text,
                                'recovery_method': cells[8].text
                            }
                            all_data.append(row_data)
                            page_data_count += 1
                    except Exception as e:
                        print(f"행 처리 중 오류: {e}")
                        continue

                print(f"페이지 {page_num}에서 {page_data_count}개 데이터 수집")

                # 다음 페이지 버튼 확인
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="DataTables_Table_0_next"]')))

                # 다음 페이지가 비활성화되어 있으면 종료
                if "disabled" in next_button.get_attribute("class"):
                    print("마지막 페이지 도달")
                    break

                # 다음 페이지로 이동
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(3)
                page_num += 1

            except Exception as e:
                print(f"페이지 {page_num} 처리 중 오류: {e}")
                break

        # DataFrame으로 변환
        df = pd.DataFrame(all_data)

        # 결과 출력
        print(f"\n=== 크롤링 완료 ===")
        print(f"총 {len(df)}개의 서울특별시 싱크홀 데이터 수집 완료")
        print(f"총 {page_num}페이지 처리")

        if len(df) > 0:
            print("\n=== 데이터 미리보기 ===")
            print(df.head())

            # 주소만 추출해서 보기
            print("\n=== 수집된 주소 목록 (처음 10개) ===")
            addresses = df['acc_loc'].tolist()
            for i, addr in enumerate(addresses[:10], 1):
                print(f"{i}. {addr}")

            # CSV 파일로 저장
            df.to_csv('seoul_sinkhole_data.csv', index=False, encoding='utf-8-sig')
            print(f"\n파일 저장 완료: seoul_sinkhole_data.csv")

            # 파일 다운로드를 위한 코드 (코랩용)
            from google.colab import files
            files.download('seoul_sinkhole_data.csv')

        return df

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

    finally:
        driver.quit()
        print("브라우저 종료 완료")

# 실행
print("=== 서울특별시 싱크홀 데이터 크롤링 시작 ===")
data = crawl_seoul_sinkhole_colab()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

브라우저 종료 완료
